In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("dgural/bdd100k", split="train[:1000]")  # Load a subset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [3]:
import os
import cv2
import time
import pandas as pd
from datetime import timedelta
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from sklearn.model_selection import train_test_split

In [4]:
# CONFIGURATION
video_dir = "/content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train"
csv_path = "/content/drive/MyDrive/driving-video-with-object-tracking/mot_labels.csv" #This file contains object name, bounding box info per frame.
output_dir = "/content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n"

# Directory setup
img_train_dir = os.path.join(output_dir, "images/train")
img_test_dir = os.path.join(output_dir, "images/test")
label_train_dir = os.path.join(output_dir, "labels/train")
label_test_dir = os.path.join(output_dir, "labels/test")

os.makedirs(img_train_dir, exist_ok=True)
os.makedirs(img_test_dir, exist_ok=True)
os.makedirs(label_train_dir, exist_ok=True)
os.makedirs(label_test_dir, exist_ok=True)

image_width, image_height = 1280, 720  # Resolution of bdd100k videos

In [ ]:
# Load CSV and extract unique classes
df = pd.read_csv(csv_path, low_memory=False)
unique_classes = sorted(df["category"].dropna().unique().tolist())
class_map = {cls: idx for idx, cls in enumerate(unique_classes)}

print("Unique Classes:", unique_classes)
print("Class Map:", class_map)

Unique Classes: ['bicycle', 'bus', 'car', 'motorcycle', 'other person', 'other vehicle', 'pedestrian', 'rider', 'trailer', 'train', 'truck']
Class Map: {'bicycle': 0, 'bus': 1, 'car': 2, 'motorcycle': 3, 'other person': 4, 'other vehicle': 5, 'pedestrian': 6, 'rider': 7, 'trailer': 8, 'train': 9, 'truck': 10}


In [ ]:
unique_classes_idd = {'animal', 'autorickshaw', 'bicycle', 'bus', 'car', 'caravan', 'motorcycle', 'person', 'rider', 'traffic light', 'traffic sign', 'trailer', 'train', 'truck', 'vehicle fallback'}

In [5]:
#create a classes set by merging unique_classes and unique_classes_idd
class_map_combined = {
    'bicycle': 0,
    'bus': 1,
    'car': 2,
    'motorcycle': 3,
    'other person': 4,
    'other vehicle': 5,
    'pedestrian': 6,
    'rider': 7,
    'trailer': 8,
    'train': 9,
    'truck': 10,
    'animal': 11,
    'autorickshaw': 12,
    'caravan': 13,
    'traffic light': 14,
    'traffic sign': 15
}

In [ ]:
# print unique value and their respective counts for category column.
print("Total number of unique classes",len(unique_classes))
print(df['category'].value_counts())

Total number of unique classes 11
category
car              2207116
pedestrian        381441
truck             149823
bus                57880
bicycle            28477
rider              20365
other vehicle      19202
motorcycle         12464
other person        4453
trailer             4103
train               1592
Name: count, dtype: int64


In [ ]:
df.head()

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo
0,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89537.0,car,False,True,False,825.173210,1003.094688,355.011547,418.198614,True
1,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89538.0,car,False,True,False,484.295612,700.461894,346.697460,424.849885,True
2,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89539.0,pedestrian,False,True,False,645.588915,663.879908,338.383372,358.337182,True
3,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89540.0,car,False,False,False,120.969977,192.471132,359.168591,409.053118,True
4,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89541.0,car,False,False,False,251.501155,315.519630,354.180139,400.739030,True


In [ ]:
print(len(os.listdir(video_dir)))

1000


In [ ]:
video_names = df["videoName"].unique()
print("Total number of unique video names",len(video_names))

Total number of unique video names 1400


In [ ]:
video = "0000f77c-6257be58.mov"

if video in set(os.listdir(video_dir)):
    print(f"Video '{video}' exists in the Train videos.")
else:
    print(f"Video '{video}' does not exist in the Train videos.")
label = "0000f77c-6257be58"
if label in video_names:
    print(f"Video '{label}' exists in the DataFrame.")
else:
    print(f"Video '{label}' does not exist in the DataFrame.")

Video '0000f77c-6257be58.mov' exists in the Train videos.
Video '0000f77c-6257be58' exists in the DataFrame.


In [ ]:
extract_frames(video, )

In [ ]:
# Split by videoName
video_names = df["videoName"].unique()
train_videos, test_videos = train_test_split(video_names, test_size=0.1, random_state=42)

In [ ]:
train_videos_subset = train_videos[:2]
test_videos_subset = test_videos[:2]

NameError: name 'train_videos' is not defined

In [ ]:
print(f"number of train videos: {len(train_videos_subset)}")
print(f"number of test videos: {len(test_videos_subset)}")

number of train videos: 2
number of test videos: 2


In [ ]:
print(f"number of train videos: {len(train_videos)}")
print(f"number of test videos: {len(test_videos)}")

number of train videos: 1260
number of test videos: 140


In [ ]:
def extract_frames(video_files, img_dir):
  num_videos = len(video_files)
  i = 0
  for video_file in video_files:

    vid_path = os.path.join(video_dir, video_file )
    vid_name = os.path.splitext(video_file)[0]
    print(f"🎬 Processing video {i + 1}/{num_videos}: {video_file}")
    print(f"vid_path: {vid_path}")

    cap = cv2.VideoCapture(vid_path)
    frame_id = 0
    saved_frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save only every 6th frame
        if (frame_id+1) % 6 == 0 or frame_id == 0:
            fname = f"{vid_name}_frame_{saved_frame_count:05d}.jpg"
            #rotated_img = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            cv2.imwrite(os.path.join(img_dir, fname), frame)
            saved_frame_count += 1
            print(f"\r🎬 Processing video {i + 1}/{num_videos}: {video_file} - Saved Frame {frame_id:05d}", end="")
        frame_id += 1

    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"\r🎬 Finished video {i + 1}/{num_videos}: {video_file} - Total frames processed: {num_frames}, Frames saved: {saved_frame_count}")

    cap.release()
    i+=1

In [ ]:
video_files = sorted([f for f in os.listdir(video_dir) if f.lower().endswith(".mov")])
num_videos = len(video_files)
print(f"\n🎞️ Found {num_videos} .mov files.")


🎞️ Found 1000 .mov files.


In [ ]:
extract_frames(video_files[:200], img_train_dir)
extract_frames(video_files[200:300], img_test_dir)

🎬 Processing video 1/200: 0000f77c-6257be58.mov
vid_path: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-6257be58.mov
🎬 Finished video 1/200: 0000f77c-6257be58.mov - Total frames processed: 1217, Frames saved: 203
🎬 Processing video 2/200: 0000f77c-62c2a288.mov
vid_path: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-62c2a288.mov
🎬 Finished video 2/200: 0000f77c-62c2a288.mov - Total frames processed: 1211, Frames saved: 202
🎬 Processing video 3/200: 0000f77c-cb820c98.mov
vid_path: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-cb820c98.mov
🎬 Finished video 3/200: 0000f77c-cb820c98.mov - Total frames processed: 1215, Frames saved: 203
🎬 Processing video 4/200: 0001542f-5ce3cf52.mov
vid_path: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/00

In [ ]:
print(f"{img_test_dir}")

/content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/images/test


In [ ]:
print(len(os.listdir(img_train_dir)))

39063


In [ ]:
print(len(os.listdir(img_test_dir)))

19880


In [ ]:
train_videos_subset = video_files[:300]
test_videos_subset = video_files[300:400]

In [ ]:
!rm -rf {img_train_dir}+"000d4f89-3bcbe37a.mov"

In [ ]:
video_file_1 = "000d35d3-41990aa4.mov"
extract_frames([video_file_1], img_train_dir)

🎬 Processing video 1/1: 000d35d3-41990aa4.mov
vid_path: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/000d35d3-41990aa4.mov
🎬 Finished video 1/1: 000d35d3-41990aa4.mov - Total frames processed: 1203, Frames saved: 201


In [ ]:
#Process all .mov videos
video_files = sorted([f for f in os.listdir(video_dir) if f.lower().endswith(".mov")])
num_videos = len(video_files)
print(f"\n🎞️ Found {num_videos} .mov files.")

# Iterate through all video files
for i, video_file in enumerate(video_files):
    vid_path = os.path.join(video_dir, video_file)
    vid_name = os.path.splitext(video_file)[0]
    print(f"\r🎬 Processing video {i + 1}/{num_videos}: {video_file}", end="")

    cap = cv2.VideoCapture(vid_path)
    frame_id = 0
    saved_frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save only every 6th frame
        if frame_id % 6 == 0:
            fname = f"{vid_name}_frame_{frame_id:05d}.jpg"
            rotated_img = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            cv2.imwrite(os.path.join(img_dir, fname), rotated_img)
            saved_frame_count += 1
            print(f"\r🎬 Processing video {i + 1}/{num_videos}: {video_file} - Saved Frame {frame_id:05d}", end="")
        frame_id += 1

    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"\r🎬 Finished video {i + 1}/{num_videos}: {video_file} - Total frames processed: {num_frames}, Frames saved: {saved_frame_count}")

    cap.release()
    break;

In [ ]:
video_files = sorted([f for f in os.listdir(video_dir) if f.lower().endswith(".mov")])
print(f"\n🎞️ Found {len(video_files)} .mov files.")


🎞️ Found 1000 .mov files.


In [ ]:
# Replace 'your_video_name' with the actual video name you want to count
video_name_to_count = '00adbb3f-7757d4ea' # Example video name

# Filter the DataFrame for the specified video name
video_df = df[df['videoName'] == video_name_to_count]

# Count the number of unique frameIndex values in the filtered DataFrame
unique_frame_count = video_df['frameIndex'].nunique()

print(f"The number of unique frame IDs for video '{video_name_to_count}' is: {unique_frame_count}")

The number of unique frame IDs for video '00adbb3f-7757d4ea' is: 202


In [ ]:
# Replace 'your_video_name' with the actual video name you want to count
video_name_to_count_1 = '000d35d3-41990aa4' # Example video name

# Filter the DataFrame for the specified video name
video_df_1 = df[df['videoName'] == video_name_to_count_1]

# Count the number of unique frameIndex values in the filtered DataFrame
unique_frame_count_1 = video_df_1['frameIndex'].nunique()

print(f"The number of unique frame IDs for video '{video_name_to_count_1}' is: {unique_frame_count_1}")

The number of unique frame IDs for video '000d35d3-41990aa4' is: 202


In [6]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


In [ ]:
model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [ ]:
!exiftool -Rotation video_files[0]

/bin/bash: line 1: exiftool: command not found


In [ ]:
# FFmpeg-based hybrid frame extraction function
import subprocess
frame_interval = 10  # Nth frame to extract alongside I-frames

def extract_video_frames(video_filename):
    input_path = os.path.join(video_dir, video_filename)
    video_name = os.path.splitext(video_filename)[0]
    base_pattern = os.path.join(img_dir, f"{video_name}_frame")

    # Extract I-frames
    i_cmd = f"""
    ffmpeg -y -skip_frame nokey -i "{input_path}" -vsync 0 -q:v 2 \
    "{base_pattern}_I_%05d.jpg"
    """

    # Extract every Nth frame
    nth_cmd = f"""
    ffmpeg -y -i "{input_path}" -vf "select='not(mod(n\\,{frame_interval}))'" \
    -vsync 0 -q:v 2 "{base_pattern}_N_%05d.jpg"
    """

    env = os.environ.copy()
    subprocess.run(i_cmd, shell=True, env=env, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(nth_cmd, shell=True, env=env, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
# Parallel frame extraction
from multiprocessing import Pool, cpu_count

# Parallel processing for frame extraction
if __name__ == "__main__":
    video_files = sorted([f for f in os.listdir(video_dir) if f.lower().endswith(".mov")])
    print(f"\n🚀 Extracting from {len(video_files)} videos using {cpu_count()} workers...")
    with Pool(processes=cpu_count()) as pool:
        pool.map(extract_video_frames, video_files)


🚀 Extracting from 1000 videos using 2 workers...


KeyboardInterrupt: 

In [ ]:
num_train_labels = 0
num_test_labels = 0
num_labels_without_match = 0

# Convert annotations to YOLO format
print("📄 Converting annotations to YOLO format...")

# Remove extension from subset video names for matching
train_videos_subset_no_ext = {os.path.splitext(v)[0] for v in video_files[:200]}
test_videos_subset_no_ext = {os.path.splitext(v)[0] for v in video_files[200:300]}

for _, row in df.iterrows():
    vname = row["videoName"]
    frame = int(row["frameIndex"])
    cls = row["category"]
    class_id = class_map.get(cls, -1)
    if class_id == -1:
        continue

    x1, x2 = float(row["box2d.x1"]), float(row["box2d.x2"])
    y1, y2 = float(row["box2d.y1"]), float(row["box2d.y2"])

    xc = ((x1 + x2) / 2) / image_width
    yc = ((y1 + y2) / 2) / image_height
    w = (x2 - x1) / image_width
    h = (y2 - y1) / image_height

    # Determine the output directory (train or test)
    if vname in train_videos_subset_no_ext:
        label_dir = label_train_dir
        num_train_labels += 1
    elif vname in test_videos_subset_no_ext:
        label_dir = label_test_dir
        num_test_labels += 1
    else:
        # This case should ideally not happen if all videos in df are in train_videos or test_videos
        num_labels_without_match += 1
        continue

    # Construct the label file path
    label_path = os.path.join(label_dir, f"{vname}_frame_{frame:05d}.txt")
    with open(label_path, "a") as f:
        f.write(f"{class_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

print("✅ Annotation conversion and saving complete.")
print(f"Total labels processed: {len(df)}")
print(f"Labels saved to train directory: {num_train_labels}")
print(f"Labels saved to test directory: {num_test_labels}")
print(f"Labels without matching video in subsets: {num_labels_without_match}")

📄 Converting annotations to YOLO format...
✅ Annotation conversion and saving complete.
Total labels processed: 2890846
Labels saved to train directory: 369410
Labels saved to test directory: 188205
Labels without matching video in subsets: 2329301


In [6]:
# Write YOLO dataset config
yaml_path = "bdd_dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""
path: {output_dir}
train: images/train
val: images/test
nc: {len(class_map_combined)}
names: {list(class_map_combined.keys())}
""")

In [10]:
# Train YOLOv11-nano
print("\n🎯 Beginning model training...")
model = YOLO("yolo11n.pt")
start = time.time()
model.train(data=yaml_path, epochs=40, imgsz=960, batch=-1, patience=20)
end = time.time()


🎯 Beginning model training...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=bdd_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, p

train: Scanning /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/labels/train.cache... 62846 images, 1372 backgrounds, 1 corrupt: 100%|██████████| 64177/64177 [00:00<?, ?it/s]

train: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/images/train/idd_highquality_16k_15-07-18-upload_0017291.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.4723      1.3486]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=960 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA A100-SXM4-40GB) 39.56G total, 13.09G reserved, 1.69G allocated, 24.78G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2592960       14.53         4.490         36.11         51.46        (1, 3, 960, 960)                    list
     2592960       29.06         4.752         31.78         36.44        (2, 3, 960, 960)                    list
     2592960       58.11         5.557          33.9         33.71        (4, 3, 960, 960)                    list
     2592960       116.2         7.262         37.68         41.63        (8, 3, 960, 960)                    list
     2592960       23

train: Scanning /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/labels/train.cache... 62846 images, 1372 backgrounds, 1 corrupt: 100%|██████████| 64177/64177 [00:00<?, ?it/s]

...

                   all      21258     204037      0.503      0.296       0.29      0.169

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/40      19.6G      1.196     0.7972      1.061        463        960: 100%|██████████| 2292/2292 [12:49<00:00,  2.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 380/380 [02:34<00:00,  2.45it/s]


                   all      21258     204037      0.499      0.298      0.291      0.169
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      31/40      19.6G      1.221     0.7939       1.07        303        960:  14%|█▎        | 310/2292 [01:45<11:13,  2.94it/s]


KeyboardInterrupt: 

In [9]:
#Resume training here
model = YOLO("/content/runs/detect/train3/weights/last.pt")
start = time.time()
model.train(resume=True,data=yaml_path, epochs=40, imgsz=960, batch=-1, patience=20)
end = time.time()

Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=bdd_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0,

train: Scanning /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/labels/train... 6824 images, 322 backgrounds, 0 corrupt:  11%|█         | 7146/64177 [28:30<3:47:29,  4.18it/s]


KeyboardInterrupt: 

In [10]:
# Evaluate model
metrics = model.val()
print("\n📊 Evaluation Results:")
print(f"mAP@0.5:   {metrics.box.map50:.4f}")
# Use the correct attributes for precision, recall, and F1 score
print(f"Precision: {metrics.box.mp:.4f}") # Access mp as an attribute
print(f"Recall:    {metrics.box.mr:.4f}")  # Access mr as an attribute
# Calculate mean F1 score if metrics.box.f1 is a list and not empty
if isinstance(metrics.box.f1, list) and len(metrics.box.f1) > 0:
    mean_f1 = sum(metrics.box.f1) / len(metrics.box.f1)
    print(f"F1 Score:  {mean_f1:.4f}")
else:
    # Handle cases where f1 might not be a list or is empty
    print(f"F1 Score:  N/A")


print(f"\n⏱ Total Time: {timedelta(seconds=end - start)}")

Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,585,272 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.0 ms, read: 0.3±0.1 MB/s, size: 228.9 KB)


val: Scanning /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/labels/test.cache... 20488 images, 770 backgrounds, 0 corrupt: 100%|██████████| 21258/21258 [00:00<?, ?it/s]

val: /content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n/images/test/idd_highquality_16k_HYD-2018-08-24_13-22-50_0007776.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1329/1329 [1:19:20<00:00,  3.58s/it]


                   all      21258     204037      0.496        0.3      0.291      0.169
               bicycle       2169       3644      0.339      0.216      0.176     0.0561
                   bus       2729       3351      0.402      0.347      0.316      0.214
                   car      19804     145139      0.708      0.659      0.684      0.372
            motorcycle       2143       4990       0.66      0.503      0.532      0.308
          other person        193        290          1          0    0.00156   0.000685
         other vehicle       1592       2248      0.471     0.0841      0.108     0.0562
            pedestrian       7534      27000      0.477      0.313      0.322      0.132
                 rider       2643       5303      0.621      0.381      0.426      0.236
               trailer          1          1          0          0          0          0
                 train         22         22          0          0          0          0
                 truc

NameError: name 'end' is not defined

In [ ]:
# Create a reverse mapping from class ID to class name
id_to_class_map = {idx: cls for cls, idx in class_map.items()}

# Now you can access the class name using the integer ID
class_name = id_to_class_map[1]
print(class_name)

bus


In [ ]:
# 🔧 Configuration
video_path = '/content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/01a8723d-89a1953a.mov'  # Input video file (can be .mov or .mp4)
output_path = 'annotated_output.mp4'  # Output video file

# 🎨 Class-specific colors for bounding boxes
CLASS_COLORS = {
    'bicycle': (0, 191, 255),        # Electric Blue – easily visible and distinct
    'bus': (255, 85, 0),             # Fire Orange – high contrast for large vehicles
    'car': (0, 255, 127),            # Spring Green – soft but stands out on urban roads
    'motorcycle': (255, 105, 180),   # Hot Pink – vivid for fast-moving bikes
    'other person': (204, 204, 0),   # Mustard Yellow – clear visibility for ambiguous figures
    'other vehicle': (128, 128, 128),# Gray – neutral for unclassified vehicle types
    'pedestrian': (50, 205, 50),     # Lime Green – associated with people and movement
    'rider': (186, 85, 211),         # Medium Orchid – bold and eye-catching for bike riders
    'trailer': (210, 105, 30),       # Chocolate – earthy tone for large towed units
    'train': (0, 0, 139),            # Dark Blue – fits the profile of rail-bound objects
    'truck': (218, 165, 32),         # Goldenrod – strong presence for heavy-duty vehicles
}

# 🎞️ Open input video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🔁 Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 🔍 Run inference on the current frame
    results = model.predict(frame, save=False)[0]

    # 🎨 Annotate each detected object with a class-specific color
    for box in results.boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_name = id_to_class_map[cls_id]
        color = CLASS_COLORS.get(cls_name, (255, 255, 255))  # Default to white if undefined

        # 🖼️ Draw the bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"{cls_name} {conf:.2f}"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # 💾 Write the annotated frame to the output video
    out.write(frame)

# 🧹 Release video objects
cap.release()
out.release()
print(f"✅ Annotated video saved to: {output_path}")



0: 384x640 (no detections), 78.5ms
Speed: 2.2ms preprocess, 78.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 2.1ms preprocess, 9.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 23.3ms
Speed: 2.8ms preprocess, 23.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.2ms
Speed: 2.7ms preprocess, 16.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.8ms
Speed: 2.2ms preprocess, 13.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.4ms
Speed: 2.5ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.3ms
Speed: 2.3ms preprocess, 16.3ms inf

In [11]:
#download runs folder to /content/drive/MyDrive/driving-video-with-object-tracking

!tar -czvf /tmp/runs_July31.tar.gz runs
!cp /tmp/runs_July31.tar.gz /content/drive/MyDrive/driving-video-with-object-tracking/

runs/
runs/detect/
runs/detect/train2/
runs/detect/train2/labels_correlogram.jpg
runs/detect/train2/train_batch0.jpg
runs/detect/train2/weights/
runs/detect/train2/weights/last.pt
runs/detect/train2/weights/best.pt
runs/detect/train2/train_batch2.jpg
runs/detect/train2/train_batch1.jpg
runs/detect/train2/args.yaml
runs/detect/train2/results.csv
runs/detect/train2/labels.jpg
runs/detect/train3/
runs/detect/train3/labels_correlogram.jpg
runs/detect/train3/train_batch0.jpg
runs/detect/train3/weights/
runs/detect/train3/weights/last.pt
runs/detect/train3/weights/best.pt
runs/detect/train3/train_batch68760.jpg
runs/detect/train3/train_batch2.jpg
runs/detect/train3/train_batch1.jpg
runs/detect/train3/args.yaml
runs/detect/train3/train_batch68761.jpg
runs/detect/train3/results.csv
runs/detect/train3/train_batch68762.jpg
runs/detect/train3/labels.jpg
runs/detect/train/
runs/detect/train/weights/
runs/detect/train/args.yaml
runs/detect/val/
runs/detect/val/val_batch1_labels.jpg
runs/detect/val/

In [7]:
#download and extract files from /content/drive/MyDrive/driving-video-with-object-tracking/runs.tar.gz
!cp /content/drive/MyDrive/driving-video-with-object-tracking/runs.tar.gz /tmp/
!tar -xzvf /tmp/runs.tar.gz -C /content/

runs/
runs/detect/
runs/detect/train2/
runs/detect/train2/args.yaml
runs/detect/train2/train_batch1.jpg
runs/detect/train2/labels_correlogram.jpg
runs/detect/train2/train_batch2.jpg
runs/detect/train2/labels.jpg
runs/detect/train2/results.csv
runs/detect/train2/weights/
runs/detect/train2/weights/best.pt
runs/detect/train2/weights/last.pt
runs/detect/train2/train_batch0.jpg
runs/detect/train3/
runs/detect/train3/args.yaml
runs/detect/train3/train_batch1.jpg
runs/detect/train3/labels_correlogram.jpg
runs/detect/train3/train_batch2.jpg
runs/detect/train3/labels.jpg
runs/detect/train3/train_batch68761.jpg
runs/detect/train3/train_batch68762.jpg
runs/detect/train3/results.csv
runs/detect/train3/weights/
runs/detect/train3/weights/best.pt
runs/detect/train3/weights/last.pt
runs/detect/train3/train_batch68760.jpg
runs/detect/train3/train_batch0.jpg
runs/detect/train/
runs/detect/train/args.yaml
runs/detect/train/weights/
